Classes required

In [6]:
from typing import Type
import pandas as pd

class TeamLadder:
    def __init__(self, team: str):
        self.team = team
        for column, init_val in ladder_columns:
            setattr(self, column, init_val)

    def add_prev_round_team_ladder(self, prev_round_team_ladder):
        for col,val in prev_round_team_ladder.items():
            self.__dict__[col] = val

    def update_home_team(self, match):
        self.played += 1
        if match.hscore > match.ascore:
            self.wins += 1
            self.prem_points += 4
        elif match.hscore == match.ascore:
            self.draws += 1
            self.prem_points += 2
        else:
            self.losses += 1    
        self.points_for += match.hscore
        self.points_against += match.ascore
        self.percentage = 100 * (self.points_for / self.points_against)

    def update_away_team(self, match):
        self.played += 1
        if match.hscore < match.ascore:
            self.wins += 1
            self.prem_points += 4
        elif match.hscore == match.ascore:
            self.draws += 1
            self.prem_points += 2
        else:
            self.losses += 1    
        self.points_for += match.ascore
        self.points_against += match.hscore
        self.percentage = 100 * (self.points_for / self.points_against)

    def update_ladder(self, match):
        """
        Update the ladder for the team based on the outcome of the game. There
        will be two possibilites - the team can be the home or the away team
        in the provided match.
        """
        if self.team == match.teams['home']:
            self.update_home_team(match)
        else:
            self.update_away_team(match)

class Ladder:
    """
    Each round object holds the ladder details for that round for each team
    """
    def __init__(self, teams_in_season):
        self.teams_in_season = teams_in_season
        self.team_ladders = {}

    def add_team_ladder(self, team_ladder):
        self.team_ladders[team_ladder.team.team] = team_ladder

class Team:
    """
    Holds team-level data for a particular match
    """
    def __init__(self, generic_team_columns, home_or_away: str):
        self.home_or_away = home_or_away
        for column in generic_team_columns:
            setattr(self, column, None)

    def add_data(self, data: pd.DataFrame):
        if self.home_or_away == 'home':
            for home_col, generic_col in home_cols_mapped.items():
                self.__dict__[generic_col] = data[home_col]
        if self.home_or_away == 'away':
            for away_col, generic_col in away_cols_mapped.items():
                self.__dict__[generic_col] = data[away_col]

class Match:
    """
    Holds data about a match, as well as an object for each team
    """
    def __init__(self, match_columns):
        self.teams = {
            'home': None,
            'away': None
        }
        for column in match_columns:
            setattr(self, column, None)

    def add_data(self, data: pd.DataFrame):
        for column in self.__dict__.keys():
            try:
                self.__dict__[column] = data[column]
            except KeyError:
                continue
    
    def add_home_team(self, team):
        self.teams['home'] = team
    
    def add_away_team(self, team):
        self.teams['away'] = team

class Round:
    """
    Contains match and ladder data for each round
    """
    def __init__(self, round_num: int):
        self.round_num = round_num
        self.matches = []
        self.bye_teams = []
        self.ladder = None

    def add_match(self, match):
        self.matches.append(match)

    def add_ladder(self, ladder):
        self.ladder = ladder

class Season:
    """
    Contains the rounds for a season, and which teams competed
    """
    def __init__(self, year: int, teams):
        self.year = year
        self.teams = teams
        self.rounds = {}
        
    def add_round(self, round_obj: Type[Round]):
        self.rounds[round_obj.round_num] = round_obj

class History:
    """
    Holds all season objects
    """
    def __init__(self):
        self.seasons = {}
        
    def add_season(self, season):
        self.seasons[season.year] = season

In [12]:
import pickle
path = '/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/ladder_data/data/history_object.pk1'
pickle_in = open(path,"rb")
ladders = pickle.load(pickle_in)

In [34]:
ladders.__dict__['seasons'][2019].__dict__['rounds'][23].__dict__['ladder'].__dict__['team_ladders']['Geelong'].__dict__




{'team': <src.match_data.generate_match_objects.Team at 0x1289d8e50>,
 'prem_points': 64,
 'losses': 6,
 'draws': 0,
 'points_for': 1984,
 'points_against': 1462,
 'played': 22,
 'position': 1,
 'percentage': 135.7045143638851,
 'wins': 16}